In [ ]:
# Blocks:
# 1 (Here): establish "global" variables. Run this once before any others and do not run again, as running it again will set the sets as empty
# 2: Organization of the files of alphabetical stat blocks into CRs. Establish the year as well as the titles of the folders. Run each time you update the data set or want to change the year analyzed
# 3: Extracts numbers from stat blocks into lists by CR. Run after running #2, as graphs rely on this filtering
# 4: Graphs for individual data sets. Run if you'd like after running #3
# 5: Graphs for the comparison of data sets. Run if you'd like after running #2 then #3 twice, once for 2014 then once for 2024

oldData = {}
newData = {}

In [5]:
#sorting
import json

editionYear = 2024

folder = 'Data' if editionYear<2024 else 'Data - 24'

crList = {"0":[],"1/8":[],"1/4":[], "1/2":[], "1":[], "2":[],"3":[],"4":[],"5":[],"6":[],"7":[],"8":[],"9":[],"10":[],
		  "11":[],"12":[],"13":[],"14":[],"15":[],"16":[],"17":[],"18":[],"19":[],"20":[],"21":[],"22":[],"23":[],"24":[],
		  "25":[],"26":[],"27":[],"28":[],"29":[],"30":[]}
crLabels = ["0","1/8","1/4", "1/2", "1", "2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18",
			"19","20","21","22","23","24","25","26","27","28","29","30"]

with open(folder+'/index.json') as json_file:
	index = json.load(json_file)

for key in index:
	file = folder+"/"+index[key]
	with open(file) as json_file1:
		monsterList = json.load(json_file1)
	for key1 in monsterList["monster"]:
		if "cr" in key1:
			if "cr" in key1["cr"] and key1['cr']['cr'] != 'null':
				crList[key1["cr"]["cr"]].append(key1)
			elif key1["cr"] != 'null':
				crList[key1["cr"]].append(key1)

In [6]:
#categorizing dataset
import json
import math

stats = ["str", "dex", "con", "int", "wis", "cha"]
longStats = {"str":"strength", "dex":"dexterity", "con":"constitution", "int":"intelligence", "wis":"wisdom", "cha":"charisma"}
conditions = ["blinded", "charmed", "deafened", "exhaustion", "frightened", "grappled", "incapacitated", "invisible", "paralyzed", "petrified", "poisoned", "prone", "restrained", "stunned", "unconscious"]
damages = ["acid", "bludgeoning", "cold","fire","force","lightning","necrotic", "piercing","poison","psychic","radiant","slashing","thunder"]
nonBPSdamages = ["acid", "cold","fire","force","lightning","necrotic","poison","psychic","radiant","thunder"]
types = ["aberration", "beast", "celestial", "construct", "dragon", "elemental", "fey", "fiend", "giant", "humanoid", "monstrosity", "ooze", "plant", "undead"]

dataList={}

def inflictionCounts(creature):
	sBool = {}
	dBool = {}
	cBool = {}
	for stat in stats:
		sBool[stat] = False
	for damage in damages:
		dBool[damage] = False
	for condition in conditions:
		cBool[condition] = False
	try:
		searchList = []
		if "trait" in creature:
			searchList+=creature["trait"]
		if "action" in creature:
			searchList+=creature["action"]
		if "bonus" in creature:
			searchList+=creature["bonus"]
		if "legendary" in creature:
			searchList+=creature["legendary"]
		#search list
		for feature in searchList:
			if not "entries" in feature:
				continue
			for entry in feature["entries"]:
				if isinstance(entry, dict):
					for subEntry in entry['items']:
						if isinstance(subEntry, dict):
							subEntryList = []
							if "entries" in subEntry:
								subEntryList+=subEntry["entries"]
							if "entry" in subEntry:
								subEntryList.append(subEntry["entry"])
							for subsubEntry in subEntryList:
								#saves
								for stat in stats:
									sBool[stat] = sBool[stat] or subsubEntry.lower().find(longStats[stat]+" saving throw")>=0 or subsubEntry.lower().find("actsave "+stat)>=0
								#damages
								for damage in damages:
									dBool[damage] = dBool[damage] or subsubEntry.lower().find(damage+" damage")>=0
								#conditions
								for condition in conditions:
									cBool[condition] = (cBool[condition] or subsubEntry.lower().find("{@condition "+condition)>=0)
						else:
							for stat in stats:
								sBool[stat] = sBool[stat] or subEntry.lower().find(longStats[stat]+" saving throw")>=0 or subEntry.lower().find("actsave "+stat)>=0
							#damages
							for damage in damages:
								dBool[damage] = dBool[damage] or subEntry.lower().find(damage+" damage")>=0
							#conditions
							for condition in conditions:
								cBool[condition] = (cBool[condition] or subEntry.lower().find("{@condition "+condition)>=0)
				else:
					#saves
					for stat in stats:
							sBool[stat] = sBool[stat] or entry.lower().find(longStats[stat]+" saving throw")>=0 or entry.lower().find("actsave "+stat)>=0
					#damages
					for damage in damages:
							dBool[damage] = dBool[damage] or entry.lower().find(damage+" damage")>=0
					#conditions
					for condition in conditions:
						cBool[condition] = (cBool[condition] or entry.lower().find("{@condition "+condition)>=0)
	except KeyError:
		print(creature["name"]+" has no actions")
	return [sBool, dBool, cBool]

for cr in crList:
	data = {"total":len(crList[cr]),"AC":[], "HP":[], 
				 #stats
				 "str":[],"dex":[],"con":[],"int":[],"wis":[],"cha":[],
				 #damage types
				 "resistances":{},
				 "immunities":{},
				 "vulnerabilities":{},
				 #coniditon types
				 "condImmunities":{},
				 #sizes
				 "T":0, "S":0, "M":0, "L":0, "H":0, "G":0,
				 "fly":0, "hover":0,
				 #offensive stats
				 "saveCounts": {},
				 "damagesInflicted": {},
				 "conditionsInflicted":{},
				 #creature types
				 "creatureTypes":{}
				}
	for stat in stats:
		data['saveCounts'][stat] = 0
	for damage in damages:
		data['damagesInflicted'][damage] = 0
		data['resistances'][damage] = 0
		data['immunities'][damage] = 0
		data['vulnerabilities'][damage] = 0
	for condition in conditions:
		data['condImmunities'][condition] = 0
		data['conditionsInflicted'][condition] = 0
	for cType in types:
		data['creatureTypes'][cType] = 0

	for monster in crList[cr]:
		#AC
		if "ac" in monster:
			if type(monster["ac"][0]) == int:
				data["AC"].append(monster["ac"][0])
			elif "ac" in monster["ac"][0]:
				data["AC"].append(monster["ac"][0]["ac"])
		else:
			data["AC"].append("-")
		#HP (avg)
		if "hp" in monster:
			if "special" in monster["hp"]:
				data["HP"].append("-")
			else:
				data["HP"].append(monster["hp"]["average"])
		else:
			data["HP"].append("-")
		#save mods
		for stat in stats:
			if "save" in monster:
				if stat in monster["save"]:
					if "PB" not in monster["save"][stat]:
						data[stat].append(int(monster["save"][stat].replace("+", "")))
				else:
					data[stat].append(math.floor((monster[stat]-10)/2))
			elif stat in monster:
				data[stat].append(math.floor((monster[stat]-10)/2))
			else:
				data[stat].append("-")
		#dmg resistances, dmg immunities, dmg vulnerabilities
		if "immune" in monster:
			for dmg in monster["immune"]:
				if type(dmg) == str:
					data["immunities"][dmg]+=1
				else:
					if "immune" in dmg:
						for dmg2 in dmg["immune"]:
							data["immunities"][dmg2]+=1
		if "resist" in monster:
			for dmg in monster["resist"]:
				if type(dmg) == str:
					data["resistances"][dmg]+=1
				else:
					if "resist" in dmg:
						for dmg2 in dmg["resist"]:
							data["resistances"][dmg2]+=1
		#vuln
		if "vulnerable" in monster:
			for dmg in monster["vulnerable"]:
				if type(dmg) == str:
					data["vulnerabilities"][dmg]+=1
				else:
					if "vulnerable" in dmg:
						for dmg2 in dmg["vulnerable"]:
							data["vulnerabilities"][dmg2]+=1
		#sizes
		if "size" in monster:
			data[monster["size"][0]]+=1
		#fly, hover
		if "speed" in monster:
			if "fly" in monster["speed"]:
				data["fly"]+=1
			if "canHover" in monster["speed"]:
				data["hover"]+=1
		else: #only dracolich doesnt bc it copies red dragon (has hover and fly, this works but only for the dataset)
			data["fly"]+=1
			data["hover"]+=1
		# offensive counts
		bools = inflictionCounts(monster)
		for stat in stats:
			if bools[0][stat]:
				data['saveCounts'][stat] += 1
		for damage in damages:
			if bools[1][damage]:
				data['damagesInflicted'][damage] += 1
		for condition in conditions:
			if bools[2][condition]:
				data['conditionsInflicted'][condition] += 1
		#creature types
		if "type" in monster:
			try:
				if "type" in monster["type"]:
					if "choose" in monster['type']['type']:
						tList = monster['type']['type']['choose']
						for t in tList:
							data['creatureTypes'][t] += 1/len(tList)
					else:
						data['creatureTypes'][monster["type"]["type"]]+=1
				else:
					data['creatureTypes'][monster["type"]]+=1
			except:
				print(cr, monster['name'], monster['type'])
	dataList.update({cr:data})
if(editionYear>=2024):
	newData = dataList
else:
	oldData = dataList

In [ ]:
#print the graphs

import json
import matplotlib.pyplot as plt
import numpy as np
import operator

colors=["r", "orange", "yellow", "g", "b", "purple", "k", "grey", "brown", "pink", "m", "cyan", "tab:blue", "lime", "goldenrod"]

def removeAll(item, list0):
	if item in list0:
		list0.remove(item)
		removeAll(item, list0)
	else:
		return list0

def plotBoxplot(data, title):
	print("Spread of Monster "+title+" by CR")
	try:
		data[0] = removeAll("-", data[0])
		fig = plt.figure(figsize=(14,9))
		ax = fig.add_axes([0,0,1,1])
		bp = ax.boxplot(data)
		for median in bp["medians"]:
			median.set(color="red", linewidth=3)
		ax.set_xticklabels(crLabels)
		plt.grid(axis="y")
		plt.show()
	except Exception as e:
		print({e})
		print(data)
		
def damagePlot(data, title):
	print("Percentage of Monsters with "+title+" Modification by CR")
	fig = plt.figure(figsize=(14,9))
	ax = fig.add_axes([0,0,1,1])
	x = np.arange(len(crLabels))
	plt.bar(x-0.2, data[0], 0.2, color='g')
	plt.bar(x, data[1], 0.2, color='r')
	plt.bar(x+0.2, data[2], 0.2, color='b')
	plt.xticks(x, crLabels)
	plt.ylim(0,1)
	plt.legend(["Vulnerability", "Resistance", "Immunity"], loc="upper left")
	plt.show()

def stackedBarPlot(dataArray, title, arrayIndexes, customColors=colors, arrayLabels=[]):
	print("Proportional Amount of "+title+" by CR")
	if len(arrayLabels)==0:
		arrayLabels = arrayIndexes
	fig = plt.figure(figsize=(14,9))
	ax = fig.add_axes([0,0,1,1])
	for index in range(len(arrayIndexes)):
		if index+1<len(arrayIndexes):
			plt.bar(crLabels, dataArray[arrayIndexes[index]], 
				bottom=[sum(item) for item in zip(*[dataArray[x] for x in arrayIndexes[index+1:]])], 
				color=customColors[index])
		else:
			plt.bar(crLabels, dataArray[arrayIndexes[index]], color=customColors[index])
	ax.legend([item.capitalize() for item in arrayLabels], loc="upper right")
	plt.xlim(-0.5, 38.5)
	plt.show()

distribution = []
ACdata = []
HPdata = []
StrData = []
DexData = []
ConData = []
IntData = []
WisData = []
ChaData = []
tinyList = []
smallList = []
mediumList = []
largeList = []
hugeList = []
gargantuanList = []
flyData = []
hoverData = []
totalSaves = []
savesInflicted = {}
for stat in stats:
	savesInflicted[stat] = []
damageData={}
damagesInflicted = {}
nonBPSdamagesInflicted = {}
for damage in damages:
	damageData[damage] = [[],[],[]]
	damagesInflicted[damage] = []
	nonBPSdamagesInflicted[damage] = []
conditionsInflicted = {}
for condition in conditions:
	conditionsInflicted[condition] = []
creatureTypes = {}
for cType in types:
	creatureTypes[cType] = []


try:
	for cr in dataList:
		distribution.append(dataList[cr]["total"])
		ACdata.append(dataList[cr]["AC"])
		HPdata.append(dataList[cr]["HP"])
		StrData.append(dataList[cr]["str"])
		DexData.append(dataList[cr]["dex"])
		ConData.append(dataList[cr]["con"])
		IntData.append(dataList[cr]["int"])
		WisData.append(dataList[cr]["wis"])
		ChaData.append(dataList[cr]["cha"])
		if dataList[cr]["total"] > 0:
			tinyList.append(dataList[cr]["T"]/dataList[cr]["total"])
			smallList.append(dataList[cr]["S"]/dataList[cr]["total"])
			mediumList.append(dataList[cr]["M"]/dataList[cr]["total"])
			largeList.append(dataList[cr]["L"]/dataList[cr]["total"])
			hugeList.append(dataList[cr]["H"]/dataList[cr]["total"])
			gargantuanList.append(dataList[cr]["G"]/dataList[cr]["total"])
			flyData.append(dataList[cr]["fly"]/dataList[cr]["total"])
			hoverData.append(dataList[cr]["hover"]/dataList[cr]["total"])
			saveTotal = 0
			for stat in stats:
				saveTotal += dataList[cr]['saveCounts'][stat]
			damageTotal = 0
			for dmgType in damages:
				damageTotal += dataList[cr]['damagesInflicted'][dmgType]
			nonBPSdamageTotal = 0
			for dmgType in nonBPSdamages:
				nonBPSdamageTotal += dataList[cr]['damagesInflicted'][dmgType]
			conditionTotal = 0
			for condition in conditions:
				conditionTotal += dataList[cr]['conditionsInflicted'][condition]
			for stat in stats:
				if saveTotal>0:
					savesInflicted[stat].append(dataList[cr]['saveCounts'][stat]/saveTotal)
				else:
					savesInflicted[stat].append(0)
			for dmgType in damages:
				damageData[dmgType][0].append(dataList[cr]["vulnerabilities"][dmgType]/dataList[cr]["total"])
				damageData[dmgType][1].append(dataList[cr]["resistances"][dmgType]/dataList[cr]["total"])
				damageData[dmgType][2].append(dataList[cr]["immunities"][dmgType]/dataList[cr]["total"])
				if damageTotal>0:
					damagesInflicted[dmgType].append(dataList[cr]['damagesInflicted'][dmgType]/damageTotal)
				else:
					damagesInflicted[dmgType].append(0)
			for dmgType in nonBPSdamages:
				if(nonBPSdamageTotal>0):
					nonBPSdamagesInflicted[dmgType].append(dataList[cr]['damagesInflicted'][dmgType]/nonBPSdamageTotal)
				else:
					nonBPSdamagesInflicted[dmgType].append(0)
			for condition in conditions:
				if(conditionTotal>0):
					conditionsInflicted[condition].append(dataList[cr]['conditionsInflicted'][condition]/conditionTotal)
				else:
					conditionsInflicted[condition].append(0)
			for cType in types:
				creatureTypes[cType].append(dataList[cr]['creatureTypes'][cType]/dataList[cr]["total"])
		else:
			tinyList.append(0)
			smallList.append(0)
			mediumList.append(0)
			largeList.append(0)
			hugeList.append(0)
			gargantuanList.append(0)
			flyData.append(0)
			hoverData.append(0)
			for stat in stats:
				savesInflicted[stat].append(0)
			for dmgType in damages:
				damageData[dmgType][0].append(0)
				damageData[dmgType][1].append(0)
				damageData[dmgType][2].append(0)
				damagesInflicted[dmgType].append(0)
			for dmgType in nonBPSdamages:
				nonBPSdamagesInflicted[dmgType].append(0)
			for condition in conditions:
				conditionsInflicted[condition].append(0)
			for cType in types:
				creatureTypes[cType].append(0)
except:
	print(cr)

print("Distribution of Creatures Amongst the CRs")
figSize = plt.figure(figsize=(14,9))
ax = figSize.add_axes([0,0,1,1])
plt.bar(crLabels, distribution, color='black')
plt.xlim(-0.5, 38.5)
plt.show()

#boxplot 1: AC
plotBoxplot(ACdata, "AC")
#boxplot 2: HP
plotBoxplot(HPdata, "Average HP")

#boxplot 3: stats
averages = [[],[],[],[],[],[]]
statData = [StrData, DexData, ConData, IntData, WisData, ChaData]
for num in range(6):
	removeAll("-", statData[num][0])
	for batch in statData[num]:
		if len(batch) >0:
			averages[num].append(np.mean(batch))
		else:
			averages[num].append(np.nan)

print("Average Saving Throw Modifiers by CR")
figAvg = plt.figure(figsize=(14,9))
ax = figAvg.add_axes([0,0,1,1])
colors = ['b', 'orange', 'g', 'r', 'purple', 'c']
ax.plot(crLabels, np.full(len(crLabels), np.nan), label=None)
for i in range(6):
	series = np.array(averages[i]).astype(np.double)
	mask = np.isfinite(averages[i])
	bp = ax.plot(np.array(crLabels)[mask], series[mask], 'o-', color=colors[i], label=longStats[stats[i]].capitalize())
ax.legend()
plt.grid(axis="x")
plt.show()

plotBoxplot(StrData, "Strength Save")
plotBoxplot(DexData, "Dexterity Save")
plotBoxplot(ConData, "Constitution Save")
plotBoxplot(IntData, "Intelligence Save")
plotBoxplot(WisData, "Wisdom Save")
plotBoxplot(ChaData, "Charisma Save")

#bar graph 1: dmg resistances, dmg immunities, dmg vulnerabilities
for dmgType in damages:
	damagePlot(damageData[dmgType], dmgType.capitalize())

#bar graph 2: sizes
print("Size Distribution of Monsters by CR")
figSize = plt.figure(figsize=(14,9))
ax = figSize.add_axes([0,0,1,1])
plt.bar(crLabels, tinyList, color='r')
plt.bar(crLabels, smallList, bottom=tinyList, color='b')
plt.bar(crLabels, mediumList,  bottom=[sum(item) for item in zip(tinyList, smallList)], color='y')
plt.bar(crLabels, largeList, bottom=[sum(item) for item in zip(tinyList, smallList, mediumList)], color='g')
plt.bar(crLabels, hugeList,  bottom=[sum(item) for item in zip(tinyList, smallList, mediumList, largeList)], color='m')
plt.bar(crLabels, gargantuanList, bottom=[sum(item) for item in zip(tinyList, smallList, mediumList, largeList, hugeList)],
		color='c')
plt.legend(["Tiny","Small","Medium","Large","Huge","Gargantuan"])
plt.xlim(-0.5, 38.5)
plt.show()

#bar graph 3: fly, hover
print("Percentage of Monsters with Flight and Hover by CR")
figFly = plt.figure(figsize=(14,9))
ax = figFly.add_axes([0,0,1,1])
x = np.arange(len(crLabels))
plt.bar(x-0.2, flyData, 0.4, color='m')
plt.bar(x+0.2, hoverData, 0.4, color='c')
plt.xticks(x, crLabels)
plt.legend(["Fly", "Hover"])
plt.show()

#stacked bar graph 1 saves inflicted
stackedBarPlot(savesInflicted, "Saving Throws Inflicted by Creature Features and Actions", stats, customColors=["r", "b", "y", "g", "m", "c"], arrayLabels=longStats.values())

#stacked bar graph 2 damages inflicted
stackedBarPlot(damagesInflicted, "Damages Inflicted by Creature Features and Actions", damages)

#stacked bar graph 3 non bps damages inflicted
stackedBarPlot(nonBPSdamagesInflicted, "Non-BPS Damages Inflicted by Creature Features and Actions", nonBPSdamages)

#stacked bar graph 4 conditions inflicted
stackedBarPlot(conditionsInflicted, "Conditions Inflicted by Creature Features and Actions", conditions)

#stacked bar graph 5 types
stackedBarPlot(creatureTypes, "Creature Types", types)

In [ ]:
#comparison of data sets
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
#graphs:
# size comparisons - 6 bar differences
# creature distribution - 14 bar differences
# flight and hover - 1 idk

# saves inflicted - 6 bar differences
# damages inflicted - 13 bar differences
# conditions inflicted - 15 bar differences

# ac - 1 box plot difference
# hp - 1 box plot difference
# saving throws - 6 box plot differences
# damage resistances - 13 idk

def barGraphDiffDisplay(title, overlap, excess14, excess24, yMin=0, yMax=0, width=14, height=9):
	print(title)
	figSize = plt.figure(figsize=(width,height))
	ax = figSize.add_axes([0,0,1,1])
	plt.bar(crLabels, overlap, color='black')
	plt.bar(crLabels, excess14, bottom=overlap, color='r')
	plt.bar(crLabels, excess24,  bottom=[sum(item) for item in zip(excess14, overlap)], color='g')
	plt.xlim(-0.5, 38.5)
	if(yMax>0 or yMin<0):
		plt.ylim(yMin,yMax)
	plt.legend(["Overlap", "Decrease from 2014", "Increase in 2024"], loc="upper right")
	plt.show()

def graphBarDiff(title, set14, set24, overrideYmin=0, overrideYmax=0):
	overlap = []
	excess14 = []
	excess24 = []
	for i in range(len(set14)):
		over=min(set14[i], set24[i])
		overlap.append(over)
		excess14.append(set14[i] - over)
		excess24.append(set24[i] - over)
	barGraphDiffDisplay(title, overlap, excess14, excess24, yMin=overrideYmin, yMax=overrideYmax)


def groupedBarChangeDisplay(title, data, nColumns=3, yMin=0, yMax=0, width=14, height=9):
	print(title)
	figSize = plt.figure(figsize=(width,height))
	ax = figSize.add_axes([0,0,1,1])
	ax.grid(which='minor', axis='x')
	x = np.arange(len(crLabels))
	if nColumns == 3:
		plt.bar(x-0.2, data[0], 0.2, color='g')
		plt.bar(x, data[1], 0.2, color='r')
		plt.bar(x+0.2, data[2], 0.2, color='b')
		plt.legend(["Vulnerability", "Resistance", "Immunity"], loc="upper left")
	else: #2
		plt.bar(x-0.2, data[0], 0.4, color='m')
		plt.bar(x+0.2, data[1], 0.4, color='c')
		plt.legend(["Fly", "Hover"])
	plt.xticks(x, crLabels)
	if(yMax>0 or yMin<0):
		plt.ylim(yMin,yMax)
	minor = x - 0.5
	plt.gca().set_xticks(minor[1:], minor=True)
	plt.grid(axis="x", which="minor", linestyle="-", color="grey")
	plt.grid(axis="y")
	plt.gca().tick_params(axis='x', which='minor', length=0)
	plt.show()

def nestedIndexDifferences(set14, set24):
	r = range(len(set14[0]))
	result = []
	for i in range(len(set14)):
		result1 = []
		for j in r:
			result1.append(set24[i][j]-set14[i][j])
		result.append(result1)
	return result

def boxPlotDiffDisplay(title, set14, set24, width=14, height=9, yMin=0,yMax=0):
	c1 = "grey"
	m1 = "red"
	c2 = "black"
	m2 = "red"
	print(title)
	boxWidth = 0.3
	figSize = plt.figure(figsize=(width,height))
	ax = figSize.add_axes([0,0,1,1])
	x = np.arange(1, len(set14)+1)
	bp14 = ax.boxplot(set14, positions=x-(boxWidth/2), widths=boxWidth,patch_artist=True,
                	boxprops=dict(facecolor=c1, edgecolor=c1),
					capprops=dict(color=c1),
					whiskerprops=dict(color=c1),
					flierprops=dict(color=c1, markeredgecolor=c1))
	for median in bp14["medians"]:
		median.set(color=m1, linewidth=3)
	bp24 = ax.boxplot(set24, positions=x+(boxWidth/2), widths=boxWidth, patch_artist=True,
                	boxprops=dict(facecolor=c2, edgecolor=c2),
					capprops=dict(color=c2),
					whiskerprops=dict(color=c2),
					flierprops=dict(color=c2, markeredgecolor=c2))
	for median in bp24["medians"]:
		median.set(color=m2, linewidth=3)
	ax.set_xticks(x)
	ax.set_xticklabels(crLabels)
	plt.grid(axis="y")
	plt.xlim(0, 35)
	if(yMax>0 or yMin<0):
		plt.ylim(yMin,yMax)
	label14 = mpatches.Patch(facecolor=c1, edgecolor=c1, label='2014')
	label24 = mpatches.Patch(facecolor=c2, edgecolor=c2, label='2024')
	plt.legend(handles=[label14, label24])
	plt.show()


def averageSaveLineGraph(year, dataArray, yMin=0, yMax=0, width=14, height=9):
	print(str(year)+" Average Saving Throw Modifiers by CR")
	figAvg = plt.figure(figsize=(width,height))
	ax = figAvg.add_axes([0,0,1,1])
	colors1 = ['b', 'orange', 'g', 'r', 'purple', 'c']
	ax.plot(crLabels, np.full(len(crLabels), np.nan), label=None)
	for i in range(len(dataArray)):
		series = np.array(dataArray[i]).astype(np.double)
		mask = np.isfinite(dataArray[i])
		bp = ax.plot(np.array(crLabels)[mask], series[mask], 'o-', color=colors1[i], label=longStats[stats[i]].capitalize())
	ax.legend(loc="upper left")
	if(yMax>0 or yMin<0):
		plt.ylim(yMin,yMax)
	plt.grid(axis="x")
	plt.show()

def saveAverages(dataset):
	result = []
	for stat in stats:
		statAvgs = []
		for l in dataset[stat]:
			if len(l)>0:
				statAvgs.append(sum(l)/len(l))
			else:
				statAvgs.append(np.nan)
		result.append(statAvgs)
	return result

def minMaxOfNestedArrays(array):
	localMin = 99999
	localMax = -99999
	for i in array:
		tuple = None
		if type(i) == list or type(i) == dict:
			tuple = minMaxOfNestedArrays(i)
		elif type(i) == str:
			tuple = minMaxOfNestedArrays(array[i])
		else:
			tuple = [i, i]

		if tuple[0] < localMin:
			localMin = tuple[0]
		if tuple[1] > localMax:
			localMax = tuple[1]
	return [localMin, localMax]



damages = ["acid","bludgeoning","cold","fire","force","lightning","necrotic","piercing","poison","psychic","radiant","slashing","thunder"]
sizes = ["T", "S", "M", "L", "H", "G"]
sizeLabels = {"T":"Tiny", "S":"Small", "M":"Medium", "L":"Large", "H":"Huge", "G":"Gargantuan"}
statAbbr = ["str", "dex", "con", "int", "wis", "cha"]
statLabels = {"str":"Strength", "dex":"Dexterity", "con":"Constitution", "int":"Intelligence", "wis":"Wisdom", "cha":"Charisma"}
types = ["aberration", "beast", "celestial", "construct", "dragon", "elemental", "fey", "fiend", "giant", "humanoid", "monstrosity", "ooze", "plant", "undead"]

ACdata14 = []
ACdata24 = []
HPdata14 = []
HPdata24 = []

statData14 = {}
statData24 = {}
for stat in statAbbr:
	statData14[stat] = []
	statData24[stat] = []

sizeLists14 = {}
sizeLists24 = {}
for size in sizes:
	sizeLists14[size] = []
	sizeLists24[size] = []

flyData14 = []
flyData24 = []
hoverData14 = []
hoverData24 = []
damageData14 = {}
damageData24 = {}
HPdata14 = []
HPdata24 = []
for damage in damages:
	damageData14[damage] = [[],[],[]]
	damageData24[damage] = [[],[],[]]

creatureTypes14 = {}
creatureTypes24 = {}
for cType in types:
	creatureTypes14[cType] = []
	creatureTypes24[cType] = []

conditionsInflicted14 = {}
conditionsInflicted24 = {}
for cond in conditions:
	conditionsInflicted14[cond] = []
	conditionsInflicted24[cond] = []

savesInflicted14 = {}
savesInflicted24 = {}
for stat in stats:
	savesInflicted14[stat] = []
	savesInflicted24[stat] = []

damageInflicted14 = {}
damageInflicted24 = {}
damageMods14 = {}
damageMods24 = {}
for damage in damages:
	damageInflicted14[damage] = []
	damageInflicted24[damage] = []
	damageMods14[damage] = [[],[],[]]
	damageMods24[damage] = [[],[],[]]

flightData14 = []
hoverData14 = []
flightData24 = []
hoverData24 = []

distribution14 = []
distribution24 = []
total14 = 0
total24 = 0
for cr in oldData:
	total14 += oldData[cr]["total"]
	total24 += newData[cr]["total"]

for cr in oldData:
	distribution14.append(oldData[cr]["total"]/total14)
	distribution24.append(newData[cr]["total"]/total24)
	ACdata14.append(oldData[cr]["AC"])
	ACdata24.append(newData[cr]["AC"])
	HPdata14.append(oldData[cr]["HP"])
	HPdata24.append(newData[cr]["HP"])
	for stat in statAbbr:
		statData14[stat].append(oldData[cr][stat])
		statData24[stat].append(newData[cr][stat])
	if oldData[cr]["total"] > 0:
		for size in sizes:
			sizeLists14[size].append(oldData[cr][size]/oldData[cr]["total"])
		conditionTotal14 = sum(oldData[cr]["conditionsInflicted"].values())
		for cond in conditions:
			conditionsInflicted14[cond].append(oldData[cr]["conditionsInflicted"][cond]/conditionTotal14)
		saveInflTotal14 = sum(oldData[cr]["saveCounts"].values())
		for stat in stats:
			savesInflicted14[stat].append(oldData[cr]["saveCounts"][stat]/saveInflTotal14)
		damageInflTotal14 = sum(oldData[cr]["damagesInflicted"].values())
		for damage in damages:
			damageInflicted14[damage].append(oldData[cr]["damagesInflicted"][damage]/damageInflTotal14)
			damageMods14[damage][0].append(oldData[cr]["vulnerabilities"][damage]/oldData[cr]["total"])
			damageMods14[damage][1].append(oldData[cr]["resistances"][damage]/oldData[cr]["total"])
			damageMods14[damage][2].append(oldData[cr]["immunities"][damage]/oldData[cr]["total"])
		for cType in types:
			creatureTypes14[cType].append(oldData[cr]["creatureTypes"][cType]/oldData[cr]["total"])
		flightData14.append(oldData[cr]["fly"]/oldData[cr]["total"])
		hoverData14.append(oldData[cr]["hover"]/oldData[cr]["total"])
	else:
		for size in sizes:
			sizeLists14[size].append(0)
		for cond in conditions:
			conditionsInflicted14[cond].append(0)
		for stat in stats:
			savesInflicted14[stat].append(0)
		for damage in damages:
			damageInflicted14[damage].append(0)
			damageMods14[damage][0].append(0)
			damageMods14[damage][1].append(0)
			damageMods14[damage][2].append(0)
		for cType in types:
			creatureTypes14[cType].append(0)
		flightData14.append(0)
		hoverData14.append(0)
	if newData[cr]["total"] > 0:
		for size in sizes:
			sizeLists24[size].append(newData[cr][size]/newData[cr]["total"])
		conditionTotal24 = sum(newData[cr]["conditionsInflicted"].values())
		for cond in conditions:
			conditionsInflicted24[cond].append(newData[cr]["conditionsInflicted"][cond]/conditionTotal24)
		saveInflTotal24 = sum(newData[cr]["saveCounts"].values())
		for stat in stats:
			savesInflicted24[stat].append(newData[cr]["saveCounts"][stat]/saveInflTotal24)
		damageInflTotal24 = sum(newData[cr]["damagesInflicted"].values())
		for damage in damages:
			damageInflicted24[damage].append(newData[cr]["damagesInflicted"][damage]/damageInflTotal24)
			damageMods24[damage][0].append(newData[cr]["vulnerabilities"][damage]/newData[cr]["total"])
			damageMods24[damage][1].append(newData[cr]["resistances"][damage]/newData[cr]["total"])
			damageMods24[damage][2].append(newData[cr]["immunities"][damage]/newData[cr]["total"])
		for cType in types:
			creatureTypes24[cType].append(newData[cr]["creatureTypes"][cType]/newData[cr]["total"])
		flightData24.append(newData[cr]["fly"]/newData[cr]["total"])
		hoverData24.append(newData[cr]["hover"]/newData[cr]["total"])
	else:
		for size in sizes:
			sizeLists24[size].append(0)
		for cond in conditions:
			conditionsInflicted24[cond].append(0)
		for stat in stats:
			savesInflicted24[stat].append(0)
		for damage in damages:
			damageInflicted24[damage].append(0)
			damageMods24[damage][0].append(0)
			damageMods24[damage][1].append(0)
			damageMods24[damage][2].append(0)
		for cType in types:
			creatureTypes24[cType].append(0)
		flightData24.append(0)
		hoverData24.append(0)

#graph printing
graphBarDiff("Distribution of Unique Stat Blocks Across CR", distribution14, distribution24)
for size in sizes:
	graphBarDiff("Proportion of "+sizeLabels[size]+" Creatures", sizeLists14[size], sizeLists24[size], overrideYmax=1)
boxPlotDiffDisplay("AC by CR", ACdata14, ACdata24, height=5)
boxPlotDiffDisplay("HP by CR", HPdata14, HPdata24, height=5)
statMinMax = minMaxOfNestedArrays([statData14, statData24])
averageSaveLineGraph(2014, saveAverages(statData14), yMin=statMinMax[0]-1, yMax=statMinMax[1]+1)
averageSaveLineGraph(2024, saveAverages(statData24), yMin=statMinMax[0]-1, yMax=statMinMax[1]+1)
for stat in statAbbr:
	boxPlotDiffDisplay(statLabels[stat]+" save by CR", statData14[stat], statData24[stat], yMin=statMinMax[0]-1, yMax=statMinMax[1]+1)
for cond in conditions:
	graphBarDiff(cond.capitalize()+" inflicted as proportion of all inflicted", conditionsInflicted14[cond], conditionsInflicted24[cond], overrideYmax=1)
for stat in stats:
	graphBarDiff(longStats[stat].capitalize()+" Saving Throws Proportionate to all Inflicted by CR", savesInflicted14[stat], savesInflicted24[stat], overrideYmax=1)
for damage in damages:
	graphBarDiff("Proportion of Damage that is "+damage.capitalize(), damageInflicted14[damage], damageInflicted24[damage], overrideYmax=1)
for cType in types:
	graphBarDiff("Proportion of Monsters that are "+cType.capitalize()+"s", creatureTypes14[cType], creatureTypes24[cType], overrideYmax=1)
groupedBarChangeDisplay("The Change in the Proportion of Creatures with a Fly Speed and Hover from 2014 to 2024", nestedIndexDifferences([flightData14, hoverData14],[flightData24, hoverData24]), 2)
damageModDifferences = {}
for damage in damages:
	damageModDifferences[damage] = nestedIndexDifferences(damageMods14[damage], damageMods24[damage])
damageModBounds = minMaxOfNestedArrays(damageModDifferences)
for damage in damages:
	groupedBarChangeDisplay("The Change in the Proportion of Creatures with "+damage+" Modification from 2014 to 2024", damageModDifferences[damage], yMin=damageModBounds[0], yMax=damageModBounds[1])